# Visualize model

In [ ]:
from finn.util.visualization import showInNetron
showInNetron("df_model_fifos.onnx")

# CPP/RTL sim
Note: not tested in this form -- but these are the order of transforms

In [ ]:
import numpy as np
import finn.core.onnx_exec as oxe

def verify_step(model, step_name):
    inp_val_sdf = np.load("cppsim_sdp_in.npy")
    iname_sdf = model.graph.input[0].name
    oname_sdf = model.graph.output[0].name
    input_dict_sdf = {iname_sdf: inp_val_sdf}
    # Execute FINN simulation
    output_dict_sdf = oxe.execute_onnx(model, input_dict_sdf, return_full_exec_context=True)
    np.save("quartznet_"+step_name+".npy", output_dict_sdf)

## CPP sim

In [ ]:
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.fpgadataflow.prepare_cppsim import PrepareCppSim
from finn.transformation.fpgadataflow.compile_cppsim import CompileCppSim
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode

model = ModelWrapper("df_model_rtlsim.onnx")
model = model.transform(PrepareCppSim())
model = model.transform(CompileCppSim())
model = model.transform(SetExecMode("cppsim"))

verify_step(model, "cppsim_out")

# RTL sim

In [ ]:
import os
from copy import deepcopy
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.fpgadataflow.annotate_cycles import AnnotateCycles
from finn.analysis.fpgadataflow.dataflow_performance import dataflow_performance
from finn.custom_op.registry import getCustomOp

from finn.transformation.fpgadataflow.create_stitched_ip import CreateStitchedIP
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim


test_fpga_part = "xcu250-figd2104-2L-e"
target_clk_ns = 7

model = ModelWrapper("df_model_rtlsim.onnx")

model = model.transform(AnnotateCycles())
perf = model_partition.analysis(dataflow_performance)
latency = perf["critical_path_cycles"]
os.environ["LIVENESS_THRESHOLD"] = str(int(latency * 5))

verify_model = deepcopy(model)
# rtlsim only supports impl_style=rtl for StreamingFIFO, ensure that
for fifo_layer in verify_model.get_nodes_by_op_type("StreamingFIFO"):
    getCustomOp(fifo_layer).set_nodeattr("impl_style", "rtl")
    getCustomOp(fifo_layer).set_nodeattr("code_gen_dir_ipgen", "")
    getCustomOp(fifo_layer).set_nodeattr("ipgen_path", "")
    getCustomOp(fifo_layer).set_nodeattr("ip_path", "")
    getCustomOp(fifo_layer).set_nodeattr("ip_vlnv", "")
# similarly for StreamingDataWidthConverter with impl_style=hls
for dwc_layer in verify_model.get_nodes_by_op_type("StreamingDataWidthConverter_Batch"):
    getCustomOp(dwc_layer).set_nodeattr("impl_style", "hls")
    getCustomOp(dwc_layer).set_nodeattr("code_gen_dir_ipgen", "")
    getCustomOp(dwc_layer).set_nodeattr("ipgen_path", "")
    getCustomOp(dwc_layer).set_nodeattr("ip_path", "")
    getCustomOp(dwc_layer).set_nodeattr("ip_vlnv", "")
verify_model = verify_model.transform(PrepareIP(test_fpga_part, target_clk_ns))
verify_model = verify_model.transform(HLSSynthIP())
verify_model = verify_model.transform(CreateStitchedIP(test_fpga_part, target_clk_ns))
verify_model = verify_model.transform(PrepareRTLSim())
verify_model.set_metadata_prop("exec_mode", "rtlsim")
verify_model.set_metadata_prop("rtlsim_trace", "whole_trace.vcd")

verify_step(verify_model, "rtlsim_out")

# Inspecting outputs

In [ ]:
import numpy as np
in_tensor = np.load("cppsim_sdp_in.npy")
in_tensor_cppsim = np.load("cppsim_out.npy", allow_pickle=True).item()['global_in']
in_tensor_rtlsim = np.load("rtlsim_out.npy", allow_pickle=True).item()['global_in']

print((in_tensor==in_tensor_cppsim).all())
print((in_tensor==in_tensor_rtlsim).all())

In [ ]:
in_tensor_cppsim = np.load("cppsim_out.npy", allow_pickle=True).item()['global_in']
in_tensor_rtlsim = np.load("rtlsim_out.npy", allow_pickle=True).item()['StreamingFIFO_39']

list_of_differences = np.argwhere(in_tensor_cppsim!=in_tensor_rtlsim)
print("Output values in total: {}\nOutput values not matching: {}".format(np.prod(np.shape(in_tensor_cppsim)),np.shape(list_of_differences)[0]))

print("Index of values that do not match: {}".format(list_of_differences))
